In [ ]:
# Step 1: Install Required Libraries
!pip install gradio requests

# Step 2: Import Libraries and Define Helper Functions
import gradio as gr
from urllib.parse import urlparse
import requests
import time
import os
import re

# Set your Hugging Face token
os.environ['HF_TOKEN'] = 'your_huggingface_token'  # Replace with your Hugging Face token

hf_token = os.environ.get("HF_TOKEN")
from gradio_client import Client
client = Client("fffiloni/safety-checker-bot", hf_token=hf_token)

def safety_check(user_prompt):
    response = client.predict(
        "consistent-character space",    # str source space
        user_prompt,    # str  in 'User sent this' Textbox component
        api_name="/infer"
    )
    return response

def parse_outputs(output):
    # Define this function according to your output parsing logic
    return output

def process_outputs(outputs):
    # Define this function according to your output processing logic
    return outputs

names = ['prompt', 'negative_prompt', 'subject', 'number_of_outputs', 'number_of_images_per_pose', 'randomise_poses', 'output_format', 'output_quality', 'seed']

def predict(request: gr.Request, *args, progress=gr.Progress(track_tqdm=True)):
    print(f"""
        —/n
        {args[0]}
        """)
    if args[0] == '' or args[0] is None:
        raise gr.Error(f"You forgot to provide a prompt.")

    try:
        is_safe = safety_check(args[0])
        print(is_safe)

        match = re.search(r'\bYes\b', is_safe)

        if match:
            status = 'Yes'
        else:
            status = None

        if status == "Yes" :
            raise gr.Error("Do not ask for such things.")
        else:
            headers = {'Content-Type': 'application/json'}
            payload = {"input": {}}
            base_url = "http://0.0.0.0:7860"
            for i, key in enumerate(names):
                value = args[i]
                if value and (os.path.exists(str(value))):
                    value = f"{base_url}/file=" + value
                if value is not None and value != "":
                    payload["input"][key] = value

            response = requests.post("http://0.0.0.0:5000/predictions", headers=headers, json=payload)

            if response.status_code == 201:
                follow_up_url = response.json()["urls"]["get"]
                response = requests.get(follow_up_url, headers=headers)
                while response.json()["status"] != "succeeded":
                    if response.json()["status"] == "failed":
                        raise gr.Error("The submission failed!")
                    response = requests.get(follow_up_url, headers=headers)
                    time.sleep(1)
            if response.status_code == 200:
                json_response = response.json()
                # If the output component is JSON return the entire output response
                if(outputs[0].get_config()["name"] == "json"):
                    return json_response["output"]
                predict_outputs = parse_outputs(json_response["output"])
                processed_outputs = process_outputs(predict_outputs)
                return tuple(processed_outputs) if len(processed_outputs) > 1 else processed_outputs[0]
            else:
                if(response.status_code == 409):
                    raise gr.Error(f"Sorry, the Cog image is still processing. Try again in a bit.")
                raise gr.Error(f"The submission failed! Error: {response.status_code}")
    except Exception as e:
        raise gr.Error(f"An error occurred: {e}")

title = "Demo for consistent-character cog image by fffiloni"
description = "Create images of a given character in different poses • running cog image by fffiloni"

css="""
#col-container{
    margin: 0 auto;
    max-width: 1400px;
    text-align: left;
}
"""
with gr.Blocks(css=css) as app:
    with gr.Column(elem_id="col-container"):
        gr.HTML(f"""
        <h2 style="text-align: center;">Consistent Character Workflow</h2>
        <p style="text-align: center;">{description}</p>
        """)

        with gr.Row():
            with gr.Column(scale=1):
                prompt = gr.Textbox(
                    label="Prompt", info='''Describe the subject. Include clothes and hairstyle for more consistency.''',
                    value="a person, dark blue suit, black tie, white pocket"
                )

                subject = gr.Image(
                    label="Subject", type="filepath"
                )

                submit_btn = gr.Button("Submit")

                with gr.Accordion(label="Advanced Settings", open=False):
                    negative_prompt = gr.Textbox(
                        label="Negative Prompt", info='''Things you do not want to see in your image''',
                        value="text, watermark, lowres, low quality, worst quality, deformed, glitch, low contrast, noisy, saturation, blurry"
                    )

                    with gr.Row():
                        number_of_outputs = gr.Slider(
                            label="Number Of Outputs", info='''The number of images to generate.''', value=2,
                            minimum=1, maximum=4, step=1,
                        )

                        number_of_images_per_pose = gr.Slider(
                            label="Number Of Images Per Pose", info='''The number of images to generate for each pose.''', value=1,
                            minimum=1, maximum=4, step=1,
                        )

                    with gr.Row():
                        randomise_poses = gr.Checkbox(
                            label="Randomise Poses", info='''Randomise the poses used.''', value=True
                        )

                        output_format = gr.Dropdown(
                            choices=['webp', 'jpg', 'png'], label="Output Format", info='''Format of the output images''', value="webp"
                        )

                    with gr.Row():
                        output_quality = gr.Number(
                            label="Output Quality", info='''Quality of the output images, from 0 to 100. 100 is best quality, 0 is lowest quality.''', value=80
                        )

                        seed = gr.Number(
                            label="Seed", info='''Set a seed for reproducibility. Random by default.''', value=None
                        )

            with gr.Column(scale=1.5):
                consistent_results = gr.Gallery(label="Consistent Results")

    inputs = [prompt, negative_prompt, subject, number_of_outputs, number_of_images_per_pose, randomise_poses, output_format, output_quality, seed]
    outputs = [consistent_results]

    submit_btn.click(
        fn=predict,
        inputs=inputs,
        outputs=outputs,
        show_api=False
    )

app.queue(max_size=12, api_open=False).launch(share=False, show_api=False, show_error=True)